In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

In [2]:
text_x_src_path = '../../Dataset/mixed_text_assignment.json'
numb_x_src_path = '../../Dataset/MixedDataSet.json'
y_src_path = '../../DataBook/Mixed_Data_Analyst.xlsx'

In [3]:
text_data = pd.read_json(text_x_src_path)
numb_data = pd.read_json(numb_x_src_path)
numb_data = numb_data.iloc[:, :-59022]

In [4]:
df_supervision = pd.read_excel(y_src_path)
plagiarised_array = df_supervision['Plagiarised'].astype(int).values

In [5]:
texts = text_data.iloc[:, :].astype(str).values.tolist()

texts = [[element if element != 'None' else '' for element in sublist] for sublist in texts]

In [6]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_sequence_len = max([len(seq) for seq in sequences])
data_vectorized = pad_sequences(sequences, maxlen=max_sequence_len)

In [7]:
X_T = data_vectorized
X_N = np.nan_to_num(numb_data.values, nan=0, copy=True).astype(int)
y = plagiarised_array
from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_resampled, y_resampled = ros.fit_resample(X_T, y)

In [8]:
X_T_train, X_T_test, y_T_train, y_T_test = train_test_split(X_T, y, test_size=0.2, random_state=32)

X_N_train, X_N_test, y_N_train, y_N_test = train_test_split(X_N, y, test_size=0.2, random_state=32)

In [9]:
textP = Perceptron()
textP.fit(X_T_train,y_T_train)

print(f"Text Training data score: {textP.score(X_T_train, y_T_train)}")
print(f"Text Test data score: {textP.score(X_T_test, y_T_test)}")

Text Training data score: 1.0
Text Test data score: 0.8888888888888888


In [10]:
numbP = Perceptron()
numbP.fit(X_N_train,y_N_train)

percepPredict = numbP.predict(X_N_test)

print(f"Text Training data score: {numbP.score(X_N_train, y_N_train)}")
print(f"Text Test data score: {numbP.score(X_N_test, y_N_test)}")

Text Training data score: 0.9528301886792453
Text Test data score: 0.9259259259259259


In [11]:
text_model = keras.models.Sequential([
    keras.layers.Input(shape=(10315,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

LOSS_FN = keras.losses.BinaryCrossentropy()

text_model.compile(optimizer='adam', loss=LOSS_FN, metrics=['accuracy'])

text_model.fit(X_T_train, y_T_train, epochs=5, validation_data=(X_T_test, y_T_test))

y_pred_text_binary_flat = (text_model.predict(X_T_test).flatten() > 0.5).astype(int)

Epoch 1/5
4/4 [==============================] - 0s 28ms/step - loss: 5.7016 - accuracy: 0.7642 - val_loss: 1.3817 - val_accuracy: 0.6667
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 1.4717 - accuracy: 0.8585 - val_loss: 0.6087 - val_accuracy: 0.8148
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 0.4671 - accuracy: 0.9151 - val_loss: 0.4618 - val_accuracy: 0.8148
Epoch 4/5
4/4 [==============================] - 0s 5ms/step - loss: 0.1136 - accuracy: 0.9528 - val_loss: 0.4407 - val_accuracy: 0.8148
Epoch 5/5
1/1 [==============================] - 0s 34ms/step


In [12]:
NPH = 32
numb_model = keras.models.Sequential([
    keras.layers.Input(shape=(16689,)),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(NPH, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

LOSS_FN = keras.losses.BinaryCrossentropy()

numb_model.compile(optimizer='adam', loss=LOSS_FN, metrics=['accuracy'])

numb_model.fit(X_N_train, y_N_train, epochs=5, validation_data=(X_N_test, y_N_test))

y_pred_numb_binary_flat = (numb_model.predict(X_N_test).flatten() > 0.5).astype(int)

Epoch 1/5
4/4 [==============================] - 0s 27ms/step - loss: 454.8592 - accuracy: 0.1698 - val_loss: 40.5545 - val_accuracy: 0.9630
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 207.4740 - accuracy: 0.8396 - val_loss: 69.5049 - val_accuracy: 0.6296
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 141.8384 - accuracy: 0.6604 - val_loss: 33.7465 - val_accuracy: 0.8148
Epoch 4/5
4/4 [==============================] - 0s 8ms/step - loss: 118.9054 - accuracy: 0.8491 - val_loss: 213.4538 - val_accuracy: 0.4815
Epoch 5/5
1/1 [==============================] - 0s 32ms/step


In [13]:
for numb, text, actual in zip(y_pred_numb_binary_flat, y_pred_text_binary_flat, y_N_test):
    print(f"Number: {numb}, Textual: {text} Actual: {actual}")

Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 1
Number: 0, Textual: 0 Actual: 1
Number: 0, Textual: 1 Actual: 1
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 0
Number: 0, Textual: 1 Actual: 0
Number: 0, Textual: 1 Actual: 1
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 1 Actual: 1
Number: 0, Textual: 0 Actual: 0
Number: 0, Textual: 0 Actual: 0


In [14]:
print(text_model.evaluate(X_T_test,y_T_test)[1])
print(numb_model.evaluate(X_N_test,y_N_test)[1])

1/1 [==============================] - 0s 12ms/step - loss: 0.5936 - accuracy: 0.7778
0.7777777910232544
1/1 [==============================] - 0s 11ms/step - loss: 39.5663 - accuracy: 0.8148
0.8148148059844971
